# Movie Recommender System

## Importing Libraries and Datasets

In [55]:
import pandas as pd
import numpy as np

In [56]:
movies = pd.read_csv('movies.csv')
credit = pd.read_csv('credits.csv')

In [57]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [58]:
 credit.head(1)['crew'].values

array(['[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cam

## Data Preprocessing

### Merging two dataframes

In [59]:
movies = movies.merge(credit, on='title')

In [60]:
movies.shape

(4809, 23)

### Extracting Important Features

In [61]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

### Finding and removing Missing Data

In [62]:
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [63]:
movies.dropna(inplace=True) # dropna removes missing rows in pandas dataframe and inplace true ensures removal is applied # directly to the original DataFrame rather than returning a new, modified copy.


In [64]:
movies.isnull().sum()

movie_id    0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

### Bringing columns like genre , keywords ,cast , crew in right format

In [65]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

* The current format is a list of dictionary we need to bring it in list like ["Action,"Adventure","SciFi"]

In [66]:
import ast # as list is in string format we need to convert it to simple list format using literal_eval of ast module

In [67]:
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name']) # append the value of ith name , value of name has key as genre
    return L

In [68]:
movies['genres'] = movies['genres'].apply(convert) # apply() method in pandas lets you apply a custom function across each row or column of a DataFrame.


In [69]:
movies.iloc[0].genres

['Action', 'Adventure', 'Fantasy', 'Science Fiction']

* Doing Same for the keywords column

In [70]:
movies['keywords'] = movies['keywords'].apply(convert) # Applying same convert to keywords

* Converting Cast column same but here we only take top 3 cast , just modify convert function a bit to extracting first 3 dict in list giving us 1st three names

In [71]:
def convert_cast(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name']) # append the value of ith name , value of name has key as genre
            counter += 1
        else:
            break
    return L

In [72]:
movies['cast'] = movies['cast'].apply(convert_cast)

In [73]:
movies.iloc[0].crew

'[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"},

* For crew we fetch only that dictionary where job name is director

In [74]:
def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name']) # append the value of ith name , value of name has key as genre
            break
    return L

In [75]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [76]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]


* Converting Overview column to lists as well so it will be easy to concate overview,cast,crew,genre etc in single list

In [77]:
movies['overview'] = movies['overview'].apply(lambda x:x.split()) # This is an anonymous function that takes an input (x) and uses the built-in str.split() method to split the string into a list of its space-separated parts.


In [78]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]


### Remove space between words in genre , cast columns etc

* like Sam Worthington becomes SamWorthington as if space is there Sam and Worthington Will be Treated as different entity by Recommender though it is name of same person same for words like Science Fiction

In [79]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])


In [80]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan]
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton]


### Combining overview genre keywords cast crew into a single column tags and making a new dataframe with movie_id , title and tags

In [81]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [82]:
new_movies = movies[['movie_id','title','tags']]

In [83]:
new_movies.head()

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."


In [84]:
new_movies['tags'] = new_movies['tags'].apply(lambda x:" ".join(x))
# if x is a list (or iterable) of strings, this function will concatenate all the strings in x into one string, with spaces in between each element. 1. It calls the join method on the string literal " ", which joins the elements of x using a space i.e " " as a separator.


C:\Users\bhavy\AppData\Local\Temp\ipykernel_13648\767954966.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_movies['tags'] = new_movies['tags'].apply(lambda x:" ".join(x))


In [85]:
new_movies.head()

,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [86]:
new_movies['tags'] = new_movies['tags'].apply(lambda x:x.lower())

C:\Users\bhavy\AppData\Local\Temp\ipykernel_13648\1386933980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_movies['tags'] = new_movies['tags'].apply(lambda x:x.lower())


In [87]:
new_movies['tags'][0]

'in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d samworthington zoesaldana sigourneyweaver jamescameron'

## Vectorization

* the most frequent words have also included words like action,actions which are same but taken different due to plurals  , we use stemming to treat these words as same

In [88]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [89]:
def stem(text):
    y= []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [90]:
new_movies['tags'] = new_movies['tags'].apply(stem)

C:\Users\bhavy\AppData\Local\Temp\ipykernel_13648\146455822.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_movies['tags'] = new_movies['tags'].apply(stem)


In [91]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english') # extract top 5000 most occuring words in tags and excluding stop_words

In [92]:
vectors = cv.fit_transform(new_movies['tags']).toarray() # it returns a sparse matrix we convert to numpy array

In [93]:
np.set_printoptions(threshold=np.inf)

In [94]:
print(cv.get_feature_names_out())

['000' '007' '10' '100' '11' '12' '13' '14' '15' '16' '17' '17th' '18'
 '18th' '18thcenturi' '19' '1910' '1920' '1930' '1940' '1944' '1950'
 '1950s' '1960' '1960s' '1970' '1970s' '1971' '1974' '1976' '1980' '1985'
 '1990' '1999' '19th' '19thcenturi' '20' '200' '2003' '2009' '20th' '21st'
 '23' '24' '25' '30' '300' '3d' '40' '50' '500' '60' '70' '80' 'aaron'
 'aaroneckhart' 'abandon' 'abduct' 'abigailbreslin' 'abil' 'abl' 'aboard'
 'abov' 'abus' 'academ' 'academi' 'accept' 'access' 'accid' 'accident'
 'acclaim' 'accompani' 'accomplish' 'account' 'accus' 'ace' 'achiev'
 'acquaint' 'act' 'action' 'actionhero' 'activ' 'activist' 'activities'
 'actor' 'actress' 'actual' 'ad' 'adam' 'adamsandl' 'adamshankman' 'adapt'
 'add' 'addict' 'adjust' 'admir' 'admit' 'adolesc' 'adopt' 'ador'
 'adrienbrodi' 'adult' 'adultanim' 'adulteri' 'adulthood' 'advanc'
 'adventur' 'adventure' 'adventures' 'advertis' 'advic' 'advis' 'affair'
 'affect' 'afghanistan' 'africa' 'african' 'africanamerican'
 'aftercredi

## Recommendation

In [95]:
from sklearn.metrics.pairwise import cosine_similarity


In [96]:
similarity = cosine_similarity(vectors)

In [97]:
similarity[1]

array([0.08346223, 1.        , 0.06063391, 0.03883678, 0.07504692,
       0.11458731, 0.02127178, 0.12927192, 0.06262243, 0.10341754,
       0.10846523, 0.07504692, 0.41400434, 0.0951303 , 0.13730876,
       0.11070186, 0.08346223, 0.2533202 , 0.06021717, 0.10979072,
       0.0613572 , 0.0579771 , 0.10502101, 0.09240617, 0.14097097,
       0.10786762, 0.10846523, 0.08783258, 0.06160411, 0.04583492,
       0.11778571, 0.10090092, 0.09007547, 0.10289915, 0.03300492,
       0.10502101, 0.07756315, 0.05564149, 0.12862394, 0.0579771 ,
       0.08134892, 0.10006256, 0.        , 0.07204382, 0.03575993,
       0.06534103, 0.11764706, 0.12478355, 0.11163607, 0.03080206,
       0.08956222, 0.04222003, 0.07756315, 0.11357771, 0.03300492,
       0.07440653, 0.07312724, 0.153393  , 0.0951303 , 0.09587062,
       0.        , 0.14002801, 0.13284223, 0.06482037, 0.03131121,
       0.020646  , 0.11891287, 0.0579771 , 0.09553946, 0.070014  ,
       0.05356716, 0.10341754, 0.11595421, 0.035007  , 0.07134

In [98]:
def recommend(movie):
    movie_index = new_movies[new_movies['title'].str.lower() == movie.lower()].index[0] # fetching index of movie from database
    similar_movies = list(enumerate(similarity[movie_index])) # if we sort movies acc to similarity we loose indexes as now 0,1 does not give similarity of movie 1 with
    # movie 2 as now positions are changed due to sort so by enumerate fn of python we make list of tuples where each tuple has movie index and similarity score with movie given it looks like [(1,0.34554).....] so it means the top most movie similar to our given is movie 1 with similarity 0.34544 , so we don't lose our access to movie index and then we sort
    sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True) # key=lambda x:x[1],reverse=True means we have to sort list on basis of 2nd quantity of tuples in list , second quantity is cosine scores in descending order

    for mov in sorted_similar_movies[1:6]:
        print(new_movies.iloc[mov[0]].title) # print similar movies by fetching them from their indexes from sorted_similar_movie list



In [99]:
recommend('gandhi')

Gandhi, My Father
Guiana 1838
The Wind That Shakes the Barley
Mr. Turner
A Passage to India


In [100]:
import pickle

In [101]:
# pickle.dump(new_movies,open('movie_list.pkl','wb'))
# pickle.dump(similarity,open('similarity.pkl','wb'))